# Asignar valor de parámetro en el notebook desde un widget de texto


In [0]:
import json

def assign_notebook_parameter_value_from_text_widget(widget_config: dict) -> None:
    """
    Crea widgets de texto en Databricks y asigna sus valores a variables de Python en el notebook.

    Esta función itera sobre un diccionario de widgets, creando cada widget de texto en Databricks y almacenando su
    valor en una variable de Python correspondiente al nombre del widget.

    Parámetros:
    - widget_config (dict): Un diccionario donde cada clave es el nombre del widget, y el valor es otro diccionario
                             que contiene 'value' (valor por defecto del widget) y 'label' (etiqueta del widget).
                             Ejemplo:
                             {
                                 'redWineUrl': {'value': 'http://example.com/redWine.csv', 'label': 'URL del Vino Tinto'},
                                 'saveDirectory': {'value': '/path/to/save', 'label': 'Directorio de Guardado' }
                             }

    Nota:
    - Esta función no devuelve ningún valor. Modifica los widgets de Databricks y asigna los valores de los widgets
      a las variables de Python correspondientes en el ámbito del notebook.
    """
    for widget_name, widget_info in widget_config.items():
        # Si el valor es una lista, convertirlo a un string
        widget_value = json.dumps(widget_info['value']) if isinstance(widget_info['value'], list) else widget_info['value']
        
        # Crear el widget de texto
        dbutils.widgets.text(widget_name, widget_value, widget_info['label'])

    # Imprimir mensaje de éxito después de crear todos los widgets
    print("Proceso completado: Todos los widgets de texto han sido creados y asignados correctamente.")

# Recuperar el valor del widget y convertirlo de nuevo a una lista
def get_widget_value_as_list(widget_name: str) -> list:
    """
    Recupera el valor de un widget de texto y lo convierte de nuevo a una lista de Python.

    Parámetros:
    - widget_name (str): Nombre del widget.

    Retorno:
    - list: Lista de valores del widget.
    """
    widget_value = dbutils.widgets.get(widget_name)
    
    # Convertir el valor del widget de vuelta a una lista
    try:
        # Intentar convertir el valor a una lista
        return json.loads(widget_value)
    except json.JSONDecodeError:
        # Si no es una lista, devolverlo como un solo valor
        return [widget_value]
    
def remove_assign_notebook_parameter_value_from_widget():
    """
    Elimina todos los widgets en el notebook de Databricks.
    """
    
    dbutils.widgets.removeAll()
    print("Todos los widgets han sido eliminados.")

In [0]:
# Definir la URL base para los archivos CSV y JSON
# Estas URLs apuntan a los repositorios en GitHub donde se encuentran los datasets
url_base_csv = 'https://raw.githubusercontent.com/JorgeCardona/data-collection-json-csv-sql/refs/heads/main/csv/wine'
url_base_json = 'https://raw.githubusercontent.com/JorgeCardona/data-collection-json-csv-sql/refs/heads/main/json/wine'

# Definir la ruta común para el directorio de guardado de los datasets
# Este es el directorio en Databricks donde se almacenarán los archivos descargados
save_directory = '/FileStore/tables'  # Sin la barra final

# Definir los nombres de los datasets en formato snake_case
# Estos son los nombres de los archivos que se utilizarán para los datasets
dataset_1_name = "winequality-red.csv"
dataset_2_name = "winequality-white.json"

# Configuración de widgets con valores y etiquetas para los datasets
# Crear un diccionario `widget_config` que asigna valores y etiquetas a los widgets utilizados
widget_config = {
    "save_directory": {"value": f"{save_directory}", "label": f"save_directory ({save_directory})"},
    "dataset_1_name": {"value": f"{dataset_1_name}", "label": f"Dataset name ({dataset_1_name})"},
    "dataset_2_name": {"value": f"{dataset_2_name}", "label": f"Dataset name ({dataset_2_name})"},
    "dataset1": {"value": f"{url_base_csv}/{dataset_1_name}", "label": f"Dataset 1 URL ({dataset_1_name})"},
    "dataset2": {"value": f"{url_base_json}/{dataset_2_name}", "label": f"Dataset 2 URL ({dataset_2_name})"},
    "saveDirectory": {"value": save_directory, "label": "Save Directory"},
    "dataset1FileLocation": {"value": f"{save_directory}/{dataset_1_name}", "label": f"Dataset 1 File Location ({dataset_1_name})"},
    "dataset2FileLocation": {"value": f"{save_directory}/{dataset_2_name}", "label": f"Dataset 2 File Location ({dataset_2_name})"},
    "fileTypeDaset1": {"value": "csv", "label": "File Type csv"},
    "fileTypeDaset2": {"value": "json", "label": "File Type json"},
    "inferSchema": {"value": "true", "label": "Infer Schema"},
    "firstRowIsHeader": {"value": "true", "label": "First Row Is Header"},
    "delimiter": {"value": ",", "label": "Delimiter"},
    "bronze": {"value": "db_bronze", "label": "Data Base Bronze"},
    "silver": {"value": "db_silver", "label": "Data Base Silver"},
    "gold": {"value": "db_gold", "label": "Data Base Gold"},
    "dataset_1_table": {"value": "red_wine", "label": "Table Name Dataset 1"},
    "dataset_2_table": {"value": "white_wine", "label": "Table Name Dataset 2"},
    "partition_columns": {"value": ["quality", "free sulfur dioxide"], "label": "Columns to partitioning datasets"}
}

# Llamar a la función `assign_notebook_parameter_value_from_text_widget` con el diccionario `widget_config`
# Esto asigna los valores y etiquetas configurados a los widgets del notebook
assign_notebook_parameter_value_from_text_widget(widget_config)

Proceso completado: Todos los widgets de texto han sido creados y asignados correctamente.


# FUNCIONES PARA DESCARGAR, LISTAR Y ELIMINAR ARCHIVOS EN UN DIRECTORIO

In [0]:
import requests
import os

# Función para descargar y guardar los archivos en databricks

def download_dataset_and_store_in_dbfs(url: str, save_path: str) -> None:
    """
    Descarga un archivo desde una URL y lo guarda en el Databricks File System (DBFS).

    Esta función descarga el contenido de un archivo desde una URL dada y lo guarda en una ruta dentro de DBFS.

    Parámetros:
    - url (str): URL desde donde se descargará el archivo.
    - save_path (str): Ruta completa en DBFS donde se guardará el archivo descargado.

    Excepciones:
    - requests.exceptions.RequestException: Si ocurre un error durante la descarga del archivo.
    - Exception: Para cualquier otro error inesperado, como problemas al guardar el archivo en DBFS.

    Ejemplo de uso:
    ```python
    download_dataset_and_store_in_dbfs("https://example.com/file.csv", "/dbfs/tmp/data/file.csv")
    ```

    El archivo será guardado en la ubicación especificada dentro de DBFS.
    """
    try:
        # Descargar el contenido del archivo
        response = requests.get(url)
        response.raise_for_status()  # Verifica si la solicitud fue exitosa

        # Usar dbutils.fs.put para guardar el contenido en DBFS
        dbutils.fs.put(save_path, response.text, overwrite=True)

        print(f"Archivo guardado correctamente en {save_path}")
    except requests.exceptions.RequestException as e:
        print(f"Error al descargar el archivo desde {url}: {e}")
    except Exception as e:
        print(f"Ocurrió un error inesperado al guardar el archivo en DBFS: {e}")


def list_files_in_directory_path(directory_path: str) -> None:
    """
    Lista los archivos dentro de un directorio especificado. Si no hay archivos, imprime un mensaje indicando
    que el directorio está vacío.

    Parámetros:
    - directory_path (str): Ruta completa del directorio cuyo contenido se desea listar.

    Retorno:
    - None: Esta función no retorna ningún valor, solo imprime el contenido del directorio o un mensaje de vacío.
    """
    files = dbutils.fs.ls(directory_path)
    if not files:
        print(f"No hay archivos en el directorio {directory_path}.")
    else:
        print(f"Archivos en el directorio {directory_path}:")
        for file in files:
            print(file.path)

def delete_all_files_in_directory_path(directory_path: str) -> None:
    """
    Elimina todos los archivos dentro de un directorio especificado. Si el directorio está vacío, imprime un mensaje 
    indicando que no hay archivos para eliminar.

    Parámetros:
    - directory_path (str): Ruta completa del directorio cuyo contenido se desea eliminar.

    Retorno:
    - None: Esta función no retorna ningún valor, solo imprime el estado de eliminación de los archivos.
    """
    # Listar todos los archivos en el directorio
    files = dbutils.fs.ls(directory_path)

    if not files:
        print(f"No hay archivos para eliminar en el directorio {directory_path}.\n")
    else:
        # Eliminar cada archivo encontrado
        print(f"Eliminando archivos en el directorio {directory_path}...\n")
        for file in files:
            dbutils.fs.rm(file.path, recurse=True)
            print(f"Archivo eliminado: {file.path}")
        
        print(f"\nTodos los archivos en el directorio {directory_path} han sido eliminados.\n")

# FUNCION PARA EJECUTAR LOS QUERIES EN SPARKSQL

In [0]:
def create_database(database_name: str) -> None:
    """
    Crea una base de datos en Databricks si no existe ya.

    Parámetros:
    - database_name (str): Nombre de la base de datos a crear.

    Retorno:
    - None

    Ejemplo de uso:
    ```python
    create_database("nombre_de_la_base_de_datos")
    ```
    """
    spark.sql(f"CREATE DATABASE IF NOT EXISTS {database_name}")
    print(f"La base de datos '{database_name}' ha sido creada exitosamente (o ya existía).")


def drop_database(database_name: str) -> None:
    """
    Elimina una base de datos en Databricks si existe.

    Parámetros:
    - database_name (str): Nombre de la base de datos que se desea eliminar.

    Retorno:
    - None

    Nota:
    Esta función elimina la base de datos especificada únicamente si existe. Para eliminar una base de datos que contenga
    tablas u objetos, utiliza la opción CASCADE dentro del comando SQL.

    Ejemplo de uso:
    ```python
    drop_database("nombre_de_la_base_de_datos")
    ```
    """
    spark.sql(f"DROP DATABASE IF EXISTS {database_name} CASCADE")
    print(f"La base de datos '{database_name}' ha sido eliminada exitosamente (si existía).")

def execute_spark_sql_query(query: str):
    """
    Ejecuta una consulta SQL en Spark.

    Parámetros:
    - query (str): Consulta SQL a ejecutar.

    Retorno:
    - pyspark.sql.dataframe.DataFrame o None: 
        - Si la consulta produce un resultado (como SELECT), retorna un DataFrame.
        - Si la consulta no produce un resultado (como UPDATE o DELETE), retorna None.

    Nota:
    La función asume que la consulta SQL está correctamente formada y que se puede ejecutar en el entorno de Spark.
    """
    try:
        print(f"Ejecutando consulta SQL:\n{query}")
        # Ejecutar el query SQL
        result = spark.sql(query)
        
        # Determinar si la consulta devuelve un DataFrame o no
        if result.isStreaming or hasattr(result, "count"):  # Verifica si tiene un resultado tangible
            print(f"La consulta SQL se ejecutó exitosamente. El número de registros obtenidos es: {result.count()}")
            return result
        else:
            print("La consulta SQL se ejecutó exitosamente. No se generó un resultado para mostrar.")
            return None
    except Exception as e:
        print(f"Ocurrió un error al ejecutar la consulta SQL:\n{query}\nError: {e}")
        return None
    
def list_tables_in_database(database_name: str) -> None:
    """
    Lista y muestra todas las tablas en una base de datos específica en Databricks.

    Parámetros:
    - database_name (str): Nombre de la base de datos de la cual se quieren listar las tablas.

    Retorno:
    - None

    Ejemplo de uso:
    ```python
    list_tables_in_database("mi_base_de_datos")
    ```
    """
    tablas = spark.sql(f"SHOW TABLES IN {database_name}")
    display(tablas)

# FUNCIONES PARA GUARDAR LOS DATAFRAMES COMOL DELTA Y PARQUET

In [0]:
from typing import List

def standardize_column_names_from_list(column_names: List[str]) -> List[str]:
    """
    Estandariza los nombres de las columnas reemplazando caracteres no alfanuméricos
    por guiones bajos ("_"). Esta transformación ayuda a evitar problemas de formato
    en el procesamiento de datos.

    Parámetros:
    - column_names (List[str]): Lista de nombres de columnas a estandarizar.

    Retorno:
    - List[str]: Nueva lista con los nombres de las columnas estandarizados.
    """
    standardized_columns = []
    
    for col_name in column_names:
        # Reemplazar caracteres no alfanuméricos por "_"
        new_col_name = ''.join([char if char.isalnum() or char == '_' else '_' for char in col_name])
        standardized_columns.append(new_col_name)
    
    print(f"Los nombres de las columnas han sido estandarizados exitosamente. Las columnas ahora usan solo caracteres alfanuméricos y guiones bajos.")
    return standardized_columns

def standardize_column_names_from_spark_dataframe(spark_dataframe: "pyspark.sql.dataframe.DataFrame") -> "pyspark.sql.dataframe.DataFrame":
    """
    Estandariza los nombres de las columnas de un DataFrame de Spark reemplazando caracteres no alfanuméricos
    por guiones bajos ("_"). Esta transformación ayuda a evitar problemas de formato en el procesamiento de datos.

    Parámetros:
    - spark_dataframe (pyspark.sql.dataframe.DataFrame): DataFrame de Spark con los nombres de columnas a estandarizar.

    Retorno:
    - pyspark.sql.dataframe.DataFrame: Nuevo DataFrame de Spark con los nombres de columnas estandarizados.
    """
    for col_name in spark_dataframe.columns:
        # Reemplazar caracteres no alfanuméricos por "_"
        new_col_name = ''.join([char if char.isalnum() or char == '_' else '_' for char in col_name])
        spark_dataframe = spark_dataframe.withColumnRenamed(col_name, new_col_name)
    
    print(f"Los nombres de las columnas han sido estandarizados exitosamente. Las columnas ahora usan solo caracteres alfanuméricos y guiones bajos.")
    return spark_dataframe

def save_dataframe_as_delta_format(spark_dataframe: "pyspark.sql.dataframe.DataFrame", path: str, partition_keys: List[str] = None) -> None:
    """
    Guarda un DataFrame de Spark en formato Delta en una ubicación específica de almacenamiento,
    opcionalmente particionado por una o más columnas.

    Parámetros:
    - spark_dataframe (DataFrame): DataFrame de Spark a guardar.
    - path (str): Ruta en el sistema de archivos donde se guardará el DataFrame en formato Delta.
    - partition_keys (List[str], opcional): Lista de nombres de columnas por las que se particionará el DataFrame.

    Retorno:
    - None

    Nota:
    Esta función sobrescribe cualquier archivo existente en la ruta especificada.
    """
    # Habilitar la fusión automática de esquemas (si es necesario)
    spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

    # Estandarizar nombres de columnas (opcional)
    spark_dataframe = standardize_column_names_from_spark_dataframe(spark_dataframe)
    partition_keys = standardize_column_names_from_list(partition_keys)
    # Guardar el DataFrame en formato Delta, particionado si se especifica
    if partition_keys:
        spark_dataframe.write.format("delta") \
            .mode("overwrite") \
            .partitionBy(partition_keys) \
            .save(path)
        print(f"El DataFrame ha sido guardado en formato Delta en '{path}', particionado por {partition_keys}")
        return partition_keys
    else:
        spark_dataframe.write.format("delta") \
            .mode("overwrite") \
            .save(path)
        print(f"El DataFrame ha sido guardado exitosamente en formato Delta en la ruta: {path}")

def save_dataframe_as_delta_table(spark_dataframe: "pyspark.sql.dataframe.DataFrame", 
                                  database_name: str, 
                                  table_name: str, 
                                  partition_by: list = None) -> None:
    """
    Guarda un DataFrame de Spark como una tabla Delta en una base de datos de Databricks.

    Parámetros:
    - spark_dataframe (pyspark.sql.dataframe.DataFrame): DataFrame de Spark a guardar.
    - database_name (str): Nombre de la base de datos donde se creará la tabla.
    - table_name (str): Nombre de la tabla Delta a crear en la base de datos.
    - partition_by (list): Lista de nombres de columnas por las cuales se debe particionar la tabla. Si no se proporciona, la tabla no será particionada.

    Retorno:
    - None

    Nota:
    Esta función sobrescribe cualquier tabla existente con el mismo nombre en la base de datos especificada.
    """
    spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

    full_table_name = f"{database_name}.{table_name}"

    # Estandarizar los nombres de las columnas (si es necesario)
    spark_dataframe = standardize_column_names_from_spark_dataframe(spark_dataframe)

    if partition_by:
        # Estandarizar los nombres de las columnas en partition_by (si es necesario)
        partition_by = standardize_column_names_from_list(partition_by)

        # Guardar la tabla particionada por las columnas especificadas
        spark_dataframe.write.format("delta") \
            .mode("overwrite") \
            .partitionBy(*partition_by) \
            .saveAsTable(full_table_name)
        print(f"El DataFrame ha sido guardado exitosamente como tabla Delta en la base de datos: {database_name}, "
              f"tabla: {table_name}, particionada por {partition_by}")
    else:
        # Guardar la tabla sin particionamiento
        spark_dataframe.write.format("delta") \
            .mode("overwrite") \
            .saveAsTable(full_table_name)
        print(f"El DataFrame ha sido guardado exitosamente como tabla Delta en la base de datos: {database_name}, "
              f"tabla: {table_name}")

def save_dataframe_as_parquet_format(spark_dataframe: "pyspark.sql.dataframe.DataFrame", 
                                     path: str, 
                                     partition_by: list = None) -> None:
    """
    Guarda un DataFrame de Spark en formato Parquet en una ubicación específica de almacenamiento,
    opcionalmente particionado por columnas específicas.

    Parámetros:
    - spark_dataframe (pyspark.sql.dataframe.DataFrame): DataFrame de Spark a guardar.
    - path (str): Ruta en el sistema de archivos donde se guardará el DataFrame en formato Parquet.
    - partition_by (list): Lista de nombres de columnas por las cuales se debe particionar el archivo Parquet.

    Retorno:
    - None

    Nota:
    Esta función sobrescribe cualquier archivo existente en la ruta especificada.
    """
    spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

    if partition_by:
        # Guardar con particionamiento
        spark_dataframe.write.format("parquet") \
            .mode("overwrite") \
            .partitionBy(partition_by) \
            .save(path)
        print(f"El DataFrame ha sido guardado en formato Parquet en '{path}', particionado por {partition_by}")
        return partition_by
    else:
        # Guardar sin particionamiento
        spark_dataframe.write.format("parquet") \
            .mode("overwrite") \
            .save(path)
        print(f"El DataFrame ha sido guardado exitosamente en formato Parquet en la ruta: {path}")


def save_dataframe_as_parquet_table(spark_dataframe: "pyspark.sql.dataframe.DataFrame", 
                                    database_name: str, 
                                    table_name: str, 
                                    partition_by: list = None) -> None:
    """
    Guarda un DataFrame de Spark como una tabla Parquet en una base de datos de Databricks.

    Parámetros:
    - spark_dataframe (pyspark.sql.dataframe.DataFrame): DataFrame de Spark a guardar.
    - database_name (str): Nombre de la base de datos donde se creará la tabla.
    - table_name (str): Nombre de la tabla Parquet a crear en la base de datos.
    - partition_by (list): Lista de nombres de columnas por las cuales se debe particionar la tabla. Si no se proporciona, la tabla no será particionada.

    Retorno:
    - None

    Nota:
    Esta función sobrescribe cualquier tabla existente con el mismo nombre en la base de datos especificada.
    """
    spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

    full_table_name = f"{database_name}.{table_name}"

    if partition_by:
        # Guardar la tabla particionada por las columnas especificadas
        spark_dataframe.write.format("parquet") \
            .mode("overwrite") \
            .partitionBy(partition_by) \
            .saveAsTable(full_table_name)
        print(f"El DataFrame ha sido guardado exitosamente como tabla Parquet en la base de datos: {database_name}, "
              f"tabla: {table_name}, particionada por {partition_by}")
    else:
        # Guardar la tabla sin particionamiento
        spark_dataframe.write.format("parquet") \
            .mode("overwrite") \
            .saveAsTable(full_table_name)
        print(f"El DataFrame ha sido guardado exitosamente como tabla Parquet en la base de datos: {database_name}, "
              f"tabla: {table_name}")

def save_dataframe_as_temp_table(spark_dataframe: "pyspark.sql.dataframe.DataFrame", 
                                 table_name: str, 
                                 partition_by: list = None) -> None:
    """
    Guarda un DataFrame de Spark como una tabla temporal en Databricks.

    Parámetros:
    - spark_dataframe (pyspark.sql.dataframe.DataFrame): DataFrame de Spark a guardar como tabla temporal.
    - table_name (str): Nombre de la tabla temporal a crear.
    - partition_by (list): Lista de columnas por las cuales particionar los datos antes de guardarlos como tabla temporal (opcional).

    Retorno:
    - None

    Nota:
    Esta función crea o reemplaza una tabla temporal, que existe solo durante la sesión activa de Spark.
    """
    spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")
    
    # Si se especifica particionamiento, particionamos el DataFrame antes de crear la tabla temporal
    if partition_by:
        spark_dataframe = spark_dataframe.repartition(*partition_by)

    # Crear la vista temporal
    spark_dataframe.createOrReplaceTempView(table_name)
    print(f"El DataFrame ha sido guardado exitosamente como tabla temporal: {table_name}")


# FUNCION PARA LEER LOS ARCHIVOS DESCARGADOS COMO SPARK DATAFRAMES

In [0]:
def load_data_to_spark_dataframe(file_location: str, 
                                 file_type: str = "csv", 
                                 infer_schema: bool = True, 
                                 first_row_is_header: bool = True, 
                                 delimiter: str = ",",
                                 is_json_array: bool = True) -> "pyspark.sql.dataframe.DataFrame":
    """
    Carga un archivo en un DataFrame de Spark, con opciones configurables para tipos de archivo, inferencia de esquema,
    encabezado y delimitador. Compatible con archivos CSV, Parquet, Delta, JSON, ORC y Avro.

    Parámetros:
    - file_location (str): La ubicación del archivo que se desea cargar.
    - file_type (str): El tipo de archivo. Valores permitidos: 'csv', 'parquet', 'delta', 'json', 'orc', 'avro'. Por defecto es 'csv'.
    - infer_schema (bool): Si se debe inferir el esquema automáticamente (aplicable solo para CSV y JSON). Por defecto es True.
    - first_row_is_header (bool): Si la primera fila debe ser usada como encabezado (aplicable solo para CSV). Por defecto es True.
    - delimiter (str): El delimitador para los archivos CSV. Por defecto es ','.
    - is_json_array (bool): Si el archivo JSON es un array de objetos (por defecto es True).

    Retorna:
    - DataFrame: El DataFrame cargado con los datos del archivo.
    """
    try:
        # Diccionario que mapea el tipo de archivo al método de carga correspondiente
        format_options = {
            "csv": {
                "format": "csv",
                "options": {
                    "inferSchema": infer_schema,
                    "header": first_row_is_header,
                    "sep": delimiter
                }
            },
            "parquet": {"format": "parquet", "options": {}},
            "delta": {"format": "delta", "options": {}},
            "json": {
                "format": "json",
                "options": {"multiline": "true" if is_json_array else "false", "inferSchema": infer_schema}  # Cambiar según si es un array
            },
            "orc": {"format": "orc", "options": {}},
            "avro": {"format": "avro", "options": {}}
        }

        # Verificar si el tipo de archivo es soportado
        if file_type not in format_options:
            raise ValueError(f"Tipo de archivo no soportado: {file_type}")

        # Obtener el formato y las opciones correspondientes
        file_format = format_options[file_type]["format"]
        options = format_options[file_type]["options"]

        # Cargar el archivo en un DataFrame
        if file_type == "json":
            # Para archivos JSON, intentamos cargarlo con las opciones definidas (multiline si es un array)
            df = spark.read.format(file_format).options(**options).json(file_location)
            
            # Verificar si el JSON es de una sola columna (_corrupt_record) y aplicar un 'flatten' si es necesario
            if df.columns == ["_corrupt_record"]:
                raise ValueError("Error al leer el archivo JSON. El archivo puede estar corrupto o no es un formato JSON válido.")
            
        else:
            # Para otros formatos de archivo, usamos el método general de carga
            df = spark.read.format(file_format).options(**options).load(file_location)

        print(f"Archivo {file_type} cargado correctamente desde {file_location}")
        return df

    except Exception as e:
        print(f"Error al cargar el archivo desde {file_location}: {e}")
        return None


# FUNCION PARA LISTAR LAS TABLAS DE LA BASE DE DATOS

In [0]:
def list_temp_tables(show_global: bool = False) -> "pyspark.sql.dataframe.DataFrame":
    """
    Lista las tablas temporales en la sesión de Spark y las devuelve como un DataFrame de Spark.

    Parámetros:
    - show_global (bool): Si es True, muestra tablas temporales globales (prefijadas con 'global_temp'). 
                          Si es False, muestra las tablas temporales locales de la sesión.

    Retorno:
    - pyspark.sql.dataframe.DataFrame: Un DataFrame con las tablas temporales disponibles.
    """
    try:
        if show_global:
            # Listar tablas temporales globales
            global_temp_tables = [
                (f"global_temp.{table.name}", "Global Temporary")
                for table in spark.catalog.listTables("global_temp")
            ]
        else:
            # Listar tablas temporales locales
            temp_tables = [
                (table.name, "Local Temporary")
                for table in spark.catalog.listTables()
                if table.isTemporary
            ]
        
        # Combinar los resultados y convertirlos a un DataFrame de Spark
        tables = global_temp_tables if show_global else temp_tables
        if tables:
            schema = ["Table", "Type"]
            display(spark.createDataFrame(tables, schema=schema))
        else:
            print("No se encontraron tablas temporales.")
            schema = ["Table", "Type"]
            display(spark.createDataFrame([], schema=schema))
    except Exception as e:
        print(f"Ocurrió un error al listar las tablas temporales: {e}")
        return None

## DESCARGAR LOS DATASETS ORIGINALES

In [0]:
# Recuperar los valores de los widgets configurados en Databricks
# Estos widgets permiten obtener URLs de descarga, nombres de archivos y rutas de almacenamiento dinámicamente

# Obtener la URL del primer dataset (vino tinto) a partir del widget "dataset1"
dataset1Url = dbutils.widgets.get("dataset1")

# Obtener la URL del segundo dataset (vino blanco) a partir del widget "dataset2"
dataset2Url = dbutils.widgets.get("dataset2")

# Obtener el directorio donde se guardarán los archivos descargados (por ejemplo, en DBFS - Databricks File System)
saveDirectory = dbutils.widgets.get("saveDirectory")

# Obtener los nombres de los archivos a partir de los widgets
# Estos nombres se utilizarán al guardar los archivos en el almacenamiento local
dataset_1_name = dbutils.widgets.get("dataset_1_name")
dataset_2_name = dbutils.widgets.get("dataset_2_name")

# Descargar y guardar los archivos de datasets en el sistema de archivos de Databricks (DBFS)
# La función `download_dataset_and_store_in_dbfs` descarga el archivo desde la URL proporcionada
# y lo guarda en la ruta especificada en DBFS
download_dataset_and_store_in_dbfs(
    dataset1Url, 
    os.path.join(saveDirectory, dataset_1_name)
)

download_dataset_and_store_in_dbfs(
    dataset2Url, 
    os.path.join(saveDirectory, dataset_2_name)
)

# Listar todos los archivos presentes en el directorio especificado
# La función `list_files_in_directory_path` muestra los archivos en la ruta `saveDirectory`
list_files_in_directory_path(saveDirectory)

Wrote 100951 bytes.
Archivo guardado correctamente en /FileStore/tables/winequality-red.csv
Wrote 1707556 bytes.
Archivo guardado correctamente en /FileStore/tables/winequality-white.json
Archivos en el directorio /FileStore/tables:
dbfs:/FileStore/tables/winequality-red.csv
dbfs:/FileStore/tables/winequality-white.json


# PROCESAMIENTO DE DATOS PARA CAPA SILVER

## LEER LOS DATASETS DESCARGADOS COMO SPARK DATAFRAMES

In [0]:
# File locations and types
# Recuperar los valores de los widgets configurados en Databricks

# Obtener el tipo de archivo (por ejemplo, "csv", "parquet", etc.) a partir de los widgets
fileTypeDaset1 = dbutils.widgets.get("fileTypeDaset1")
fileTypeDaset2 = dbutils.widgets.get("fileTypeDaset2")

# Determinar si se debe inferir el esquema automáticamente
inferSchema = dbutils.widgets.get("inferSchema")

# Indicar si la primera fila contiene los nombres de las columnas (cabecera)
firstRowIsHeader = dbutils.widgets.get("firstRowIsHeader")

# Obtener el delimitador de columnas en el archivo (por ejemplo, "," para CSV)
delimiter = dbutils.widgets.get("delimiter")

# Obtener los nombres de los datasets a partir de los widgets
dataset_1_name = dbutils.widgets.get("dataset_1_name")
dataset_2_name = dbutils.widgets.get("dataset_2_name")

# Obtener el directorio donde se guardan los archivos de datos
saveDirectory = dbutils.widgets.get("saveDirectory")

# Construir las rutas completas a los archivos de datos utilizando el directorio y nombres de datasets
dataset_1_location = f"{saveDirectory}/{dataset_1_name}"
dataset_2_location = f"{saveDirectory}/{dataset_2_name}"

# Cargar el primer dataset en un DataFrame de Spark utilizando los parámetros obtenidos
red_wine_df = load_data_to_spark_dataframe(
    dataset_1_location, fileTypeDaset1, inferSchema, firstRowIsHeader, delimiter
)

# Persistir el DataFrame en memoria y disco para optimizar las operaciones posteriores
red_wine_df.persist()

# Cargar el segundo dataset en otro DataFrame de Spark
white_wine_df = load_data_to_spark_dataframe(
    dataset_2_location, fileTypeDaset2, inferSchema, firstRowIsHeader, delimiter
)

# Cachear el DataFrame en memoria para mejorar el rendimiento si se reutiliza
white_wine_df.cache()

# Mostrar las primeras 5 filas del DataFrame `red_wine_df` en el notebook
display(red_wine_df.limit(5))

# Mostrar las primeras 5 filas del DataFrame `white_wine_df` en el notebook
display(white_wine_df.limit(5))


Archivo csv cargado correctamente desde /FileStore/tables/winequality-red.csv
Archivo json cargado correctamente desde /FileStore/tables/winequality-white.json


fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.2,0.68,9.8,5
7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.997,3.26,0.65,9.8,5
11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.998,3.16,0.58,9.8,6
7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


alcohol,chlorides,citric acid,density,fixed acidity,free sulfur dioxide,pH,quality,residual sugar,sulphates,total sulfur dioxide,volatile acidity
8.8,0.045,0.36,1.001,7.0,45.0,3.0,6,20.7,0.45,170.0,0.27
9.5,0.049,0.34,0.994,6.3,14.0,3.3,6,1.6,0.49,132.0,0.3
10.1,0.05,0.4,0.9951,8.1,30.0,3.26,6,6.9,0.44,97.0,0.28
9.9,0.058,0.32,0.9956,7.2,47.0,3.19,6,8.5,0.4,186.0,0.23
9.9,0.058,0.32,0.9956,7.2,47.0,3.19,6,8.5,0.4,186.0,0.23


## DEFINICION DE VARIABLES

In [0]:
# Obtener los valores de las columnas de partición desde un widget en Databricks.
# Se asume que la función `get_widget_value_as_list` convierte el valor del widget en una lista.
partition_columns = get_widget_value_as_list("partition_columns")

# Recuperar el directorio de guardado a partir del valor del widget.
# Esto indica dónde se guardarán o leerán los archivos dentro del entorno de Databricks.
saveDirectory = dbutils.widgets.get("saveDirectory")

# Obtener el nombre de la base de datos (por ejemplo, capa "silver") configurado en los widgets.
# Las bases de datos en Databricks suelen organizarse en diferentes capas como "bronze", "silver", y "gold".
database = dbutils.widgets.get("silver")

# Asignar valores específicos a las columnas de partición.
# Estos valores se usarán para filtrar o seleccionar particiones en los DataFrames.
partition_column_1_value = '3'      # Ejemplo de valor para la primera columna de partición
partition_column_2_value = '5.0'    # Ejemplo de valor para la segunda columna de partición

### GUARDA EL DATAFRAME EN FORMATO DELTA

In [0]:
# Definir las rutas para guardar los DataFrames en formato Delta
# Utilizando el directorio de guardado (saveDirectory) y nombres descriptivos
deltaPath1 = f"{saveDirectory}/Delta_dataset1"
deltaPath2 = f"{saveDirectory}/Delta_dataset2"

# Guardar el DataFrame `red_wine_df` en formato Delta en la ruta especificada (deltaPath1)
# Se utiliza partición basada en las columnas definidas en `partition_columns`
# La función `save_dataframe_as_delta_format` devuelve la lista de columnas de partición usadas
partition_columns_delta_1 = save_dataframe_as_delta_format(
    red_wine_df, 
    deltaPath1, 
    partition_columns
)

# Listar todos los archivos y carpetas creados en el directorio `deltaPath1`
# Esto verifica que el DataFrame se haya guardado correctamente en formato Delta
list_files_in_directory_path(deltaPath1)
print()  # Imprimir una línea en blanco para mayor claridad en la salida

# Guardar el DataFrame `white_wine_df` en formato Delta en la ruta especificada (deltaPath2)
partition_columns_delta_2 = save_dataframe_as_delta_format(
    white_wine_df, 
    deltaPath2, 
    partition_columns
)

# Listar todos los archivos y carpetas creados en el directorio `deltaPath2`
list_files_in_directory_path(deltaPath2)
print()  # Imprimir una línea en blanco para separar la salida

# Listar los archivos dentro de una partición específica en `Delta_dataset1`
# Se construye la ruta de la partición utilizando los valores de las columnas de partición
list_files_in_directory_path(
    f'{deltaPath1}/{partition_columns_delta_1[0]}={partition_column_1_value}/'
    f'{partition_columns_delta_1[1]}={partition_column_2_value}'
)
print()  # Línea en blanco para separar la salida

# Listar los archivos dentro de una partición específica en `Delta_dataset2`
list_files_in_directory_path(
    f'{deltaPath2}/{partition_columns_delta_2[0]}={partition_column_1_value}/'
    f'{partition_columns_delta_2[1]}={partition_column_2_value}'
)


Los nombres de las columnas han sido estandarizados exitosamente. Las columnas ahora usan solo caracteres alfanuméricos y guiones bajos.
Los nombres de las columnas han sido estandarizados exitosamente. Las columnas ahora usan solo caracteres alfanuméricos y guiones bajos.
El DataFrame ha sido guardado en formato Delta en '/FileStore/tables/Delta_dataset1', particionado por ['quality', 'free_sulfur_dioxide']
Archivos en el directorio /FileStore/tables/Delta_dataset1:
dbfs:/FileStore/tables/Delta_dataset1/_delta_log/
dbfs:/FileStore/tables/Delta_dataset1/quality=3/
dbfs:/FileStore/tables/Delta_dataset1/quality=4/
dbfs:/FileStore/tables/Delta_dataset1/quality=5/
dbfs:/FileStore/tables/Delta_dataset1/quality=6/
dbfs:/FileStore/tables/Delta_dataset1/quality=7/
dbfs:/FileStore/tables/Delta_dataset1/quality=8/

Los nombres de las columnas han sido estandarizados exitosamente. Las columnas ahora usan solo caracteres alfanuméricos y guiones bajos.
Los nombres de las columnas han sido estandari

### GUARDA EL DATAFRAME EN FORMATO PARQUET

In [0]:
# Definir las rutas para guardar los DataFrames en formato Parquet
# Utilizando el directorio de guardado (saveDirectory) y nombres descriptivos
parquetPath1 = f"{saveDirectory}/Parquet_dataset1"
parquetPath2 = f"{saveDirectory}/Parquet_dataset2"

# Guardar el DataFrame `red_wine_df` en formato Parquet en la ruta especificada (parquetPath1)
# Utiliza las columnas de partición definidas en `partition_columns`
# La función `save_dataframe_as_parquet_format` devuelve la lista de columnas de partición usadas
partition_columns_parquet_1 = save_dataframe_as_parquet_format(
    red_wine_df, 
    parquetPath1, 
    partition_columns
)

# Listar todos los archivos y carpetas creados en el directorio `parquetPath1`
# Esto verifica que el DataFrame se haya guardado correctamente en formato Parquet
list_files_in_directory_path(parquetPath1)
print()  # Imprimir una línea en blanco para separar la salida

# Guardar el DataFrame `white_wine_df` en formato Parquet en la ruta especificada (parquetPath2)
partition_columns_parquet_2 = save_dataframe_as_parquet_format(
    white_wine_df, 
    parquetPath2, 
    partition_columns
)

# Listar todos los archivos y carpetas creados en el directorio `parquetPath2`
list_files_in_directory_path(parquetPath2)
print()  # Imprimir una línea en blanco para mayor claridad en la salida

# Listar los archivos dentro de una partición específica en `Parquet_dataset1`
# La ruta de la partición se construye usando los nombres y valores de las columnas de partición
list_files_in_directory_path(
    f'{parquetPath1}/{partition_columns_parquet_1[0]}={partition_column_1_value}/'
    f'{partition_columns_parquet_1[1]}={partition_column_2_value}'
)
print()  # Línea en blanco para separar la salida

# Listar los archivos dentro de una partición específica en `Parquet_dataset2`
list_files_in_directory_path(
    f'{parquetPath2}/{partition_columns_parquet_2[0]}={partition_column_1_value}/'
    f'{partition_columns_parquet_2[1]}={partition_column_2_value}'
)

El DataFrame ha sido guardado en formato Parquet en '/FileStore/tables/Parquet_dataset1', particionado por ['quality', 'free sulfur dioxide']
Archivos en el directorio /FileStore/tables/Parquet_dataset1:
dbfs:/FileStore/tables/Parquet_dataset1/_SUCCESS
dbfs:/FileStore/tables/Parquet_dataset1/quality=3/
dbfs:/FileStore/tables/Parquet_dataset1/quality=4/
dbfs:/FileStore/tables/Parquet_dataset1/quality=5/
dbfs:/FileStore/tables/Parquet_dataset1/quality=6/
dbfs:/FileStore/tables/Parquet_dataset1/quality=7/
dbfs:/FileStore/tables/Parquet_dataset1/quality=8/

El DataFrame ha sido guardado en formato Parquet en '/FileStore/tables/Parquet_dataset2', particionado por ['quality', 'free sulfur dioxide']
Archivos en el directorio /FileStore/tables/Parquet_dataset2:
dbfs:/FileStore/tables/Parquet_dataset2/_SUCCESS
dbfs:/FileStore/tables/Parquet_dataset2/quality=3/
dbfs:/FileStore/tables/Parquet_dataset2/quality=4/
dbfs:/FileStore/tables/Parquet_dataset2/quality=5/
dbfs:/FileStore/tables/Parquet_dat

### GUARDA EL DATAFRAME COMO TABLA EN FORMATO DELTA

In [0]:
# Llama a la función `create_database` con el nombre de la base de datos que fue previamente almacenado en la variable `database`.
# Esta función tiene como objetivo crear una base de datos en el entorno de Spark si aún no existe.
create_database(database)

La base de datos 'db_silver' ha sido creada exitosamente (o ya existía).


In [0]:
# Definir el nombre de la tabla Delta para el primer dataset (vino tinto)
tabla_dataset1 = f'dataset1_delta'

# Definir la ruta donde se almacenará la tabla Delta para `dataset1`
tableParquetPath1 = f"{saveDirectory}/{tabla_dataset1}"

# Definir el nombre de la tabla Delta para el segundo dataset (vino blanco)
tabla_dataset2 = f'dataset2_delta'

# Definir la ruta donde se almacenará la tabla Delta para `dataset2`
tableParquetPath2 = f"{saveDirectory}/{tabla_dataset2}"

# Guardar el DataFrame `red_wine_df` como tabla Delta en la base de datos especificada
# Se utiliza partición según las columnas definidas en `partition_columns`
save_dataframe_as_delta_table(red_wine_df, database, tabla_dataset1, partition_columns)
print()  # Línea en blanco para separar la salida en la consola

# Guardar el DataFrame `white_wine_df` como tabla Delta en la misma base de datos
save_dataframe_as_delta_table(white_wine_df, database, tabla_dataset2, partition_columns)
print()  # Línea en blanco para separar la salida en la consola

# Listar todas las tablas disponibles en la base de datos especificada
list_tables_in_database(database)


Los nombres de las columnas han sido estandarizados exitosamente. Las columnas ahora usan solo caracteres alfanuméricos y guiones bajos.
Los nombres de las columnas han sido estandarizados exitosamente. Las columnas ahora usan solo caracteres alfanuméricos y guiones bajos.
El DataFrame ha sido guardado exitosamente como tabla Delta en la base de datos: db_silver, tabla: dataset1_delta, particionada por ['quality', 'free_sulfur_dioxide']

Los nombres de las columnas han sido estandarizados exitosamente. Las columnas ahora usan solo caracteres alfanuméricos y guiones bajos.
Los nombres de las columnas han sido estandarizados exitosamente. Las columnas ahora usan solo caracteres alfanuméricos y guiones bajos.
El DataFrame ha sido guardado exitosamente como tabla Delta en la base de datos: db_silver, tabla: dataset2_delta, particionada por ['quality', 'free_sulfur_dioxide']



database,tableName,isTemporary
db_silver,dataset1_delta,false
db_silver,dataset2_delta,false


# VERIFICAR LAS COLUMNAS Y EL TIPO DE DATOS DE LA TABLA

In [0]:
query = f'''
DESCRIBE EXTENDED {database}.{tabla_dataset1}
'''
display(execute_spark_sql_query(query))

query = f'''
DESCRIBE EXTENDED {database}.{tabla_dataset2}
'''
display(execute_spark_sql_query(query))

Ejecutando consulta SQL:

DESCRIBE EXTENDED db_silver.dataset1_delta

La consulta SQL se ejecutó exitosamente. El número de registros obtenidos es: 30


col_name,data_type,comment
fixed_acidity,double,null
volatile_acidity,double,null
citric_acid,double,null
residual_sugar,double,null
chlorides,double,null
free_sulfur_dioxide,double,null
total_sulfur_dioxide,double,null
density,double,null
pH,double,null
sulphates,double,null


Ejecutando consulta SQL:

DESCRIBE EXTENDED db_silver.dataset2_delta

La consulta SQL se ejecutó exitosamente. El número de registros obtenidos es: 30


col_name,data_type,comment
alcohol,double,null
chlorides,double,null
citric_acid,double,null
density,double,null
fixed_acidity,double,null
free_sulfur_dioxide,double,null
pH,double,null
quality,bigint,null
residual_sugar,double,null
sulphates,double,null


### GUARDA EL DATAFRAME COMO TABLA EN FORMATO PARQUET

In [0]:
# Definir el nombre de la tabla Parquet para el primer dataset (vino tinto)
tabla_dataset1 = f'dataset1_parquet'

# Definir la ruta donde se almacenará la tabla Parquet para `dataset1`
tableParquetPath1 = f"{saveDirectory}/{tabla_dataset1}"

# Definir el nombre de la tabla Parquet para el segundo dataset (vino blanco)
tabla_dataset2 = f'dataset2_parquet'

# Definir la ruta donde se almacenará la tabla Parquet para `dataset2`
tableParquetPath2 = f"{saveDirectory}/{tabla_dataset2}"

# Guardar el DataFrame `red_wine_df` como tabla Parquet en la base de datos especificada
# Se utiliza partición según las columnas definidas en `partition_columns`
save_dataframe_as_parquet_table(red_wine_df, database, tabla_dataset1, partition_columns)
print()  # Línea en blanco para separar la salida en la consola

# Guardar el DataFrame `white_wine_df` como tabla Parquet en la misma base de datos
save_dataframe_as_parquet_table(white_wine_df, database, tabla_dataset2, partition_columns)
print()  # Línea en blanco para separar la salida en la consola

# Listar todas las tablas disponibles en la base de datos especificada
list_tables_in_database(database)

El DataFrame ha sido guardado exitosamente como tabla Parquet en la base de datos: db_silver, tabla: dataset1_parquet, particionada por ['quality', 'free sulfur dioxide']

El DataFrame ha sido guardado exitosamente como tabla Parquet en la base de datos: db_silver, tabla: dataset2_parquet, particionada por ['quality', 'free sulfur dioxide']



database,tableName,isTemporary
db_silver,dataset1_delta,false
db_silver,dataset1_parquet,false
db_silver,dataset2_delta,false
db_silver,dataset2_parquet,false


## GUARDAR EL DATAFRAME COMO TABLAS TEMPORALES

In [0]:
# Definir el nombre de la tabla temporal para el primer dataset (vino tinto)
tabla_dataset1 = f'dataset1_temp'

# Definir la ruta donde se almacenará la tabla temporal para `dataset1`
tableParquetPath1 = f"{saveDirectory}/{tabla_dataset1}"

# Definir el nombre de la tabla temporal para el segundo dataset (vino blanco)
tabla_dataset2 = f'dataset2_temp'

# Definir la ruta donde se almacenará la tabla temporal para `dataset2`
tableParquetPath2 = f"{saveDirectory}/{tabla_dataset2}"

# Guardar el DataFrame `red_wine_df` como una tabla temporal local en el entorno de Spark
# Se utiliza partición según las columnas definidas en `partition_columns`
save_dataframe_as_temp_table(red_wine_df, tabla_dataset1, partition_columns)
print()  # Línea en blanco para separar la salida en la consola

# Guardar el DataFrame `white_wine_df` como una tabla temporal local en el entorno de Spark
save_dataframe_as_temp_table(white_wine_df, tabla_dataset2, partition_columns)
print()  # Línea en blanco para separar la salida en la consola

# Listar todas las tablas temporales locales en el entorno de Spark
list_temp_tables()

# Listar todas las tablas temporales globales en el entorno de Spark
# El parámetro `show_global=True` muestra tablas temporales globales si existen
list_temp_tables(show_global=True)

# Listar todas las tablas disponibles en la base de datos especificada
list_tables_in_database(database)

El DataFrame ha sido guardado exitosamente como tabla temporal: dataset1_temp

El DataFrame ha sido guardado exitosamente como tabla temporal: dataset2_temp



Table,Type
dataset1_temp,Local Temporary
dataset2_temp,Local Temporary


Table,Type
global_temp.dataset1_temp,Global Temporary
global_temp.dataset2_temp,Global Temporary


database,tableName,isTemporary
db_silver,dataset1_delta,false
db_silver,dataset1_parquet,false
db_silver,dataset2_delta,false
db_silver,dataset2_parquet,false
,dataset1_temp,true
,dataset2_temp,true


# ELIMINAS LOS PARAMETROS DEL NOTEBOOK Y LOS ARCHIVOS CREADOS

In [0]:
# Descartar los DataFrames en memoria para liberar espacio en caché
# `unpersist()` elimina el almacenamiento en caché de los DataFrames previamente cargados
red_wine_df.unpersist()
white_wine_df.unpersist()

# Recuperar el nombre de la base de datos desde el widget "silver"
# Esta base de datos puede estar asociada a un entorno de datos de tipo "silver"
database = dbutils.widgets.get("silver")

# Eliminar la base de datos especificada por la variable `database`
# Esto eliminará la base de datos y todos los objetos asociados a ella
drop_database(database)

# Eliminar todos los archivos en el directorio especificado por `saveDirectory`
# Esto puede incluir los archivos de datos previamente guardados
delete_all_files_in_directory_path(saveDirectory)

# Eliminar el valor asignado al parámetro del notebook en el widget
# Esto limpia el valor del parámetro del widget, lo que puede ser útil si ya no es necesario
remove_assign_notebook_parameter_value_from_widget()

La base de datos 'db_silver' ha sido eliminada exitosamente (si existía).
Eliminando archivos en el directorio /FileStore/tables...

Archivo eliminado: dbfs:/FileStore/tables/Delta_dataset1/
Archivo eliminado: dbfs:/FileStore/tables/Delta_dataset2/
Archivo eliminado: dbfs:/FileStore/tables/Parquet_dataset1/
Archivo eliminado: dbfs:/FileStore/tables/Parquet_dataset2/
Archivo eliminado: dbfs:/FileStore/tables/winequality-red.csv
Archivo eliminado: dbfs:/FileStore/tables/winequality-white.json

Todos los archivos en el directorio /FileStore/tables han sido eliminados.

Todos los widgets han sido eliminados.
